In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

spark.range(5).show()


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [4]:
csv_path = "/sparkdata/bmw.csv"

df = spark.read.csv(csv_path, header=True, inferSchema=True)

df.show()

+---------+----+-----+------------+-------+--------+---+-----+----------+
|    model|year|price|transmission|mileage|fuelType|tax|  mpg|engineSize|
+---------+----+-----+------------+-------+--------+---+-----+----------+
| 5 Series|2014|11200|   Automatic|  67068|  Diesel|125| 57.6|       2.0|
| 6 Series|2018|27000|   Automatic|  14827|  Petrol|145| 42.8|       2.0|
| 5 Series|2016|16000|   Automatic|  62794|  Diesel|160| 51.4|       3.0|
| 1 Series|2017|12750|   Automatic|  26676|  Diesel|145| 72.4|       1.5|
| 7 Series|2014|14500|   Automatic|  39554|  Diesel|160| 50.4|       3.0|
| 5 Series|2016|14900|   Automatic|  35309|  Diesel|125| 60.1|       2.0|
| 5 Series|2017|16000|   Automatic|  38538|  Diesel|125| 60.1|       2.0|
| 2 Series|2018|16250|      Manual|  10401|  Petrol|145| 52.3|       1.5|
| 4 Series|2017|14250|      Manual|  42668|  Diesel| 30| 62.8|       2.0|
| 5 Series|2016|14250|   Automatic|  36099|  Diesel| 20| 68.9|       2.0|
|       X3|2017|15500|      Manual|  7

In [11]:
manual_cars = df.where(df.transmission == "Manual")

#manual_cars.show()


In [8]:
manual_cars.write.csv("/sparkdata/manual_cars.csv")

In [9]:
df.write.parquet("/sparkdata/test_parquet/bmw.parquet")

In [10]:
df_from_parquet = spark.read.parquet("/sparkdata/test_parquet/bmw.parquet")

df_from_parquet.show()

+---------+----+-----+------------+-------+--------+---+-----+----------+
|    model|year|price|transmission|mileage|fuelType|tax|  mpg|engineSize|
+---------+----+-----+------------+-------+--------+---+-----+----------+
| 5 Series|2014|11200|   Automatic|  67068|  Diesel|125| 57.6|       2.0|
| 6 Series|2018|27000|   Automatic|  14827|  Petrol|145| 42.8|       2.0|
| 5 Series|2016|16000|   Automatic|  62794|  Diesel|160| 51.4|       3.0|
| 1 Series|2017|12750|   Automatic|  26676|  Diesel|145| 72.4|       1.5|
| 7 Series|2014|14500|   Automatic|  39554|  Diesel|160| 50.4|       3.0|
| 5 Series|2016|14900|   Automatic|  35309|  Diesel|125| 60.1|       2.0|
| 5 Series|2017|16000|   Automatic|  38538|  Diesel|125| 60.1|       2.0|
| 2 Series|2018|16250|      Manual|  10401|  Petrol|145| 52.3|       1.5|
| 4 Series|2017|14250|      Manual|  42668|  Diesel| 30| 62.8|       2.0|
| 5 Series|2016|14250|   Automatic|  36099|  Diesel| 20| 68.9|       2.0|
|       X3|2017|15500|      Manual|  7

In [12]:
df_from_parquet.createOrReplaceTempView("CARS")
spark.sql("SELECT * FROM CARS").show()

+---------+----+-----+------------+-------+--------+---+-----+----------+
|    model|year|price|transmission|mileage|fuelType|tax|  mpg|engineSize|
+---------+----+-----+------------+-------+--------+---+-----+----------+
| 5 Series|2014|11200|   Automatic|  67068|  Diesel|125| 57.6|       2.0|
| 6 Series|2018|27000|   Automatic|  14827|  Petrol|145| 42.8|       2.0|
| 5 Series|2016|16000|   Automatic|  62794|  Diesel|160| 51.4|       3.0|
| 1 Series|2017|12750|   Automatic|  26676|  Diesel|145| 72.4|       1.5|
| 7 Series|2014|14500|   Automatic|  39554|  Diesel|160| 50.4|       3.0|
| 5 Series|2016|14900|   Automatic|  35309|  Diesel|125| 60.1|       2.0|
| 5 Series|2017|16000|   Automatic|  38538|  Diesel|125| 60.1|       2.0|
| 2 Series|2018|16250|      Manual|  10401|  Petrol|145| 52.3|       1.5|
| 4 Series|2017|14250|      Manual|  42668|  Diesel| 30| 62.8|       2.0|
| 5 Series|2016|14250|   Automatic|  36099|  Diesel| 20| 68.9|       2.0|
|       X3|2017|15500|      Manual|  7

In [15]:
model_counts = spark.sql("SELECT model, COUNT(*) FROM CARS GROUP BY model")

model_counts.show()

model_counts.write.csv("/sparkdata/model_counts.csv")

+---------+--------+
|    model|count(1)|
+---------+--------+
|       X7|      55|
|       Z3|       7|
|       X6|     106|
|       X2|     288|
| 3 Series|    2443|
| 2 Series|    1229|
|       X4|     179|
|       X5|     468|
|       M4|     125|
|       i3|      43|
| 4 Series|     995|
| 8 Series|      39|
| 6 Series|     108|
|       M3|      27|
|       M2|      21|
|       Z4|     108|
|       i8|      17|
| 7 Series|     106|
|       X3|     551|
| 1 Series|    1969|
+---------+--------+
only showing top 20 rows



In [16]:
df.write.partitionBy("fuelType", "transmission").parquet("/sparkdata/partitioned/bmw.parquet")